# Computing the best gamma correction parameter to apply model from one image to another

Training the model to detect moldic pores in the original image turns out to not transfer very well to other images because the blue tone of the impregnated resin is different in both images to one another. For that reason, we apply a gamma correction to the training image so that the blue tone of the trained image best fits the tone of blue resin in the new image, for which we want to make predictions.

In this notebook, we explore multiple gamma factors and the resulting detection of moldic pores in the new image, such that we can establish a methodology to detect the best gamma correction.

In [ ]:
import os
print(os.getcwd())

In [ ]:
import os
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F

from pre_sal_ii.models.EncoderNN import EncoderNN
from pre_sal_ii.training.porosity_supervised import train_porosity_supervised, apply_model
from pre_sal_ii.improc import scale_image_and_save


In [ ]:
image_name = "ML-tste_original"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)
path = f"../out/classificada_01/{image_name}_25.jpg"

image_name_classes = "ML-tste_classidicada"
path_classes = f"../data/classificada_01/{image_name_classes}.jpg"
scale_image_and_save(path_classes, "../out/classificada_01/", 25)
path_classes = f"../out/classificada_01/{image_name_classes}_25.jpg"

image_name_target = "122.20_jpeg_escal"
path_target = f"../data/thin_sections/{image_name_target}.jpg"
scale_image_and_save(path_target, "../out/thin_sections_4x/", 25)
path_target = f"../out/thin_sections_4x/{image_name_target}_25.jpg"

In [ ]:
np.arange(0.3, 0.71, 0.05)

In [ ]:
for gamma in np.arange(0.3, 0.71, 0.1):
    os.makedirs("../models/varying_gamma/", exist_ok=True)
    model_path = f"../models/varying_gamma/supervised-1-gamma={gamma}.bin"
    os.makedirs("../out/varying_gamma_model_preds/", exist_ok=True)
    result_image_path = f"../out/varying_gamma_model_preds/sup_pred_{image_name}_gamma={gamma}.jpg"
    if not os.path.isfile(result_image_path):
        if os.path.isfile(model_path):
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = EncoderNN().to(device)
            model.load_state_dict(torch.load(model_path))
            model.eval()
        else:
            model = train_porosity_supervised(
                path, path_classes,
                save_model_path=model_path,
                gamma=gamma)
        img_result = apply_model(path_target, model)
        cv2.imwrite(result_image_path, img_result)


In [ ]:
for gamma in np.arange(0.3, 0.71, 0.1):
    print(f"{gamma:.2f}")

In [ ]:
for gamma in np.arange(0.3, 0.71, 0.1):
    result_image_path = f"../out/varying_gamma_model_preds/sup_pred_{image_name}_gamma={gamma}.jpg"
    inputImage = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)
    count_whites = sum(sum(inputImage > 255*0.9))
    print(f"gamma={gamma}, count_whites={count_whites}")
